# EigenFold

In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
import torch
from proteome.models.folding.omegafold.modeling import OmegaFoldForSequenceEmbedding

In [4]:
sequence = 'MAAHKGAEHHHKAAEHHEQAAKHHHAAAEHHEKGEHEQAAHHADTAYAHHKHAEEHAAQAAKHDAEHHAPKPH'

In [5]:
embedder = OmegaFoldForSequenceEmbedding()

In [6]:
node_repr, edge_repr = embedder.embed(sequence)

In [10]:
from dataclasses import dataclass

@dataclass
class Args:
    model_dir: str
    ckpt: str
    splits: str
    split_key: str = None
    inf_mols: int = 1000
    wandb: str = None
    num_workers: int = None
    ode: bool = False
    elbo: bool = False
    alpha: float = 0
    beta: float = 1
    num_samples: int = 1
    inf_step: float = 0.5
    elbo_step: float = 0.2
    inf_type: str = 'rate'
    max_len: int = 1024
    inf_Hf: float = None
    inf_kmin: int = None
    inf_tmin: int = None
    inf_cutoff: int = None
    embeddings_dir: str = None
    pdb_dir: str = None
    embeddings_key: str = None

In [18]:
@dataclass
class ModelConfig:
    sde_a: float = 3/(3.8**2)
    sde_b: float = 0
    resi_conv_layers: int = 6
    resi_ns: int = 32
    resi_nv: int = 4
    resi_ntps: int = 16
    resi_ntpv: int = 4
    resi_fc_dim: int = 128
    resi_pos_emb_dim: int = 16
    lin_nf: int = 1
    lin_self: bool = False
    attention: bool = False
    sh_lmax: int = 2
    order: int = 1
    t_emb_dim: int = 32
    t_emb_type: str = 'sinusoidal'
    radius_emb_type: str = 'gaussian'
    radius_emb_dim: int = 50
    radius_emb_max: float = 50
    tmin: float = 0.001
    tmax: float = 1e6
    no_radius_sqrt: bool = False
    parity: bool = True
    lm_edge_dim: int = 128
    lm_node_dim: int = 256
    no_edge_embs: bool = False

In [12]:
ckpt = torch.hub.load_state_dict_from_url(
    "https://github.com/bjing2016/EigenFold/raw/master/pretrained_model/epoch_7.pt", map_location="cpu", file_name="eigenfold_epoch_7.pt"
)

Downloading: "https://github.com/bjing2016/EigenFold/raw/master/pretrained_model/epoch_7.pt" to /home/conradry71/.cache/torch/hub/checkpoints/eigenfold_epoch_7.pt
100%|███████████████████████████████████████████████| 6.63M/6.63M [00:00<00:00, 84.5MB/s]


In [15]:
from proteome.models.design.eigenfold.score_model import get_model

In [19]:
ef_model = get_model(ModelConfig())

/home/conradry71/micromamba/envs/proteome/lib/python3.10/site-packages/torch/jit/_check.py:172: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


In [20]:
ef_model.load_state_dict(ckpt['model'], strict=True)

<All keys matched successfully>

In [ ]:
import os, yaml, torch, wandb
import numpy as np
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from utils.logging import get_logger
logger = get_logger(__name__)
from utils.dataset import get_loader
import pandas as pd
from model import get_model
from utils.inference import inference_epoch

In [ ]:
val_loader = get_loader(args, None, splits, mode=inf_args.split_key, shuffle=False)
samples, log = inference_epoch(args, model, val_loader.dataset, device=device, pdbs=True, elbo=inf_args.elbo)

means = {key: np.mean(log[key]) for key in log if key != 'path'}
logger.info(f"Inference epoch {ep}: len {len(log['rmsd'])} MEANS {means}")

for samp in samples:
    samp.pdb.write(os.path.join(inf_args.model_dir, inf_name, samp.path.split('/')[-1] + f".{samp.copy}.anim.pdb"), reverse=True)
    samp.pdb.clear().add(samp.Y).write(
        os.path.join(inf_args.model_dir, inf_name, samp.path.split('/')[-1] + f".{samp.copy}.pdb"))